In [10]:
from bs4 import BeautifulSoup
import re
import os.path
my_path = os.path.abspath('')

In [11]:
#Data cleaning approach:

#First identify the table of content by regex to get mention of "Item" in <Table> <tbody> tag(perhaps extract the table of content)
#or search <a href="#....." util the mention of Item16, signautre
#extract all the href in <a> attribute in the table of content (ex. href1, href2, href3 …)

#Second have algorithm to identify <a name="href1"> and save the content between href1 and href2, href2 and href3, …

#Third, clean the html tags in the contents 

In [13]:
def load_html(path):
    return BeautifulSoup(open(path,"r"), 'html.parser')

def soup_get_text(soup):
    print(soup.get_text())

In [14]:
#First extract the location of the table of content 
# use regex to identify Signature/Item 16 and get everything above
#only works with the text and items in table of content contains href#if only in the page number, doesn't work

def remove_duplicates_in_list(List):
    return list(dict.fromkeys(List))
def get_table_of_content_links1(soup):
    tag = soup.find('a',string=re.compile("Financial Statement Schedules"))
    tag_item = soup.find('a',string=re.compile("Item 15"))
    if tag==None and tag_item!=None:
        tag = tag_item
    
    links = list()
    links.append(tag.get('href')[1:])
    for elements in tag.previous_elements:
        if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
            #print(elements)
            links.append(elements.get('href')[1:])
    
    links.reverse()
    return remove_duplicates_in_list(links)
    #print_list(links)

#
def get_table_of_content_links2(soup):
    links = list()
    for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
        if link.get('href')!=None and link.get('href')[0]=='#':
            links.append(link.get('href')[1:])
    return links
# or identify the signature link and use find_previous method to extract:extract()?

def print_list(lists):
    for list in lists:
        print(list)


In [15]:
#unable to format tables
def get_all_sections(sections, links):
    for i in range(len(links)):
        if i<len(links)-1:
            sections.append(get_section(links[i],links[i+1]))
        else:
            sections.append(get_last_section(links[i]))

def get_section(link1, link2):#need to add "id"
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('name') and elements['name']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if elements.name=='a' and elements.has_attr('id') and elements['id']==link2:
                    break 
                if str(elements)[0] != '<':
                    section_str = section_str + str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

def get_last_section(link1):
    section_str=""
    for link in soup.find_all('a'):
        if link.has_attr('name') and link['name']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
        elif link.has_attr('id') and link['id']==link1:
            for elements in link.next_elements:
                if str(elements)[0] != '<':
                    section_str = section_str + str(elements)
                    #print(repr(elements))
            #Sections = Sections + str(elements)
            break
    return section_str

In [22]:
def file_reader(path):
    files = []
    for r,d,f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r,file))
                
    return files

In [16]:
MSFT_1_path='sec_edgar_filings/MSFT/10-K/0001193125-15-272806.txt'
MSFT_2_path="sec_edgar_filings/MSFT/10-K/0001564590-18-019062.txt"
APPL_1_path='sec_edgar_filings/AAPL/10-K/0000320193-19-000119.txt'
APPL_2_path='sec_edgar_filings/AAPL/10-K/0000320193-18-000145.txt'
VISA_1_path='sec_edgar_filings/V/10-K/0001403161-18-000055.txt'
VISA_2_path='sec_edgar_filings/V/10-K/0001403161-19-000050.txt'
GOOG_1_path='sec_edgar_filings/GOOG/10-K/0001652044-19-000004.txt'
GOOG_2_path='sec_edgar_filings/GOOG/10-K/0001652044-20-000008.txt'
BLK_1_path='sec_edgar_filings/BLK/10-K/0001564590-19-005479.txt'
BLK_2_path='sec_edgar_filings/BLK/10-K/0001564590-20-007807.txt'
BA_1_path='sec_edgar_filings/BA/10-K/0000012927-19-000010.txt'
BA_2_path='sec_edgar_filings/BA/10-K/0000012927-20-000014.txt'
FORD_1_path='sec_edgar_filings/F/10-K/0000037996-20-000010.txt'
FORD_2_path='sec_edgar_filings/F/10-K/0000037996-19-000012.txt'

soup=load_html(MSFT_1_path)
#soup_get_text(soup)
#visa, msft, goog, blk work
#, ba_1_path uses <a id="links">
#ford dont work cuz href is at the page number in toc


In [23]:
files = [];
files = file_reader(my_path+"/sec_edgar_filings")
files
soup=load_html(files[0])

In [24]:
links = get_table_of_content_links1(soup)
print_list(links)
print(len(links))

sE45D95EC302C53F3999A16F238922716
s8FA93AA667C0535398014FA54B397439
s107EA6B564A4575DA42B4A73ACD9C71A
s09BC0FC06350577DAA71D344EA07A929
s91DF3ECA7CDE54B296324D6C925A3A64
sE96BF62935E55A508C134F5B2F0CA85E
sD33472E8BE865C43851A1C3B62FDB2D2
s441770BCD3195825A459C905D90A255A
sF8C8F387D3165D459768F3D870FAEA13
sEDE9BAA1C9E05AB4843082ACD6A97FE3
s86531024D4355D39AEEBA31406A0E758
s37F84E3E4CEC530F9DE46181A4344271
sDBCC0D7FC5D05F49A572F9AA0627E992
s6B01DE738B9E59FF8C3F8CFE9C9E0B79
sA9A243DA245A56F5AC25705E9A3D277F
s05D204CD87755D688D28C56362110879
s046E02B1765C5FC98134C77D8F55508A
sF9CD356C29FA5DC18DCAC248942C78F9
sD1B069C378005F3AB1606C1AADA06047
s196AADBFDD535C31B8B9CD690EED64EC
sC8A90ABF36F05F8CBA046AD164ED5378
sFEE386E5673257028D26E7FE59FD1E21
s49498D0CD5C959CA84C417CEA46A6953
sC8BE3318CBF05904A4195064A5A404FE
24


In [25]:
sections=list()
get_all_sections(sections,links)

In [26]:
print(sections[1])
text_file = open("sample_KG.txt", "w")
text_file.write(sections[1])
text_file.close()

Item 1.BusinessCompany BackgroundThe Company designs, manufactures and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The Company’s fiscal year is the 52- or 53-week period that ends on the last Saturday of September. The Company is a California corporation established in 1977.ProductsiPhoneiPhone® is the Company’s line of smartphones based on its iOS operating system. In September 2019, the Company introduced three new iPhones: iPhone 11, iPhone 11 Pro and iPhone 11 Pro Max.MacMac® is the Company’s line of personal computers based on its macOS® operating system. During 2019, the Company released a new version of MacBook Air® and a new Mac mini®, and introduced an updated Mac Pro®, which is expected to be available in the fall of 2019.iPadiPad® is the Company’s line of multi-purpose tablets. iPad is based on the Company’s iPadOS™ operating system, which was introduced during 2019. Also during 2019, the Company relea

In [10]:
"""
#get table of content

"identify the mention of Signatures"
tag = soup.find('a',string=re.compile("Signatures"))
#print(tag)
links = list()
for elements in tag.previous_elements:
    if elements.name=='a' and elements.has_attr('href') and elements.get('href')[0]=='#':
        #print(elements)
        links.append(elements.get('href')[1:])
        
links.reverse()
#print_list(links)
"""

'\n#get table of content\n\n"identify the mention of Signatures"\ntag = soup.find(\'a\',string=re.compile("Signatures"))\n#print(tag)\nlinks = list()\nfor elements in tag.previous_elements:\n    if elements.name==\'a\' and elements.has_attr(\'href\') and elements.get(\'href\')[0]==\'#\':\n        #print(elements)\n        links.append(elements.get(\'href\')[1:])\n        \nlinks.reverse()\n#print_list(links)\n'

In [11]:
#load the html
"""
soup = BeautifulSoup(open("/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/MSFT/10-K/0001564590-19-027952.txt"), 'html.parser')
#print(soup.prettify())
print(soup.get_text())
"""



'\nsoup = BeautifulSoup(open("/Users/davidren/Desktop/Education/KTH/2019-2020/2020 Spring/Master Thesis/Data/sec_edgar_filings/MSFT/10-K/0001564590-19-027952.txt"), \'html.parser\')\n#print(soup.prettify())\nprint(soup.get_text())\n'

In [12]:
"""
#get all href from the table of content 
Links = list()
for link in soup.find_all('a'):
    #link.get('href')
    #print(link.get('href'))
    #print(link.get('name'))
    if link.get('href')!=None and link.get('href')[0]=='#':
        Links.append(link.get('href')[1:])
for link in Links:
    print(link)
        
    #remove the '#' and store them in a list
"""

"\n#get all href from the table of content \nLinks = list()\nfor link in soup.find_all('a'):\n    #link.get('href')\n    #print(link.get('href'))\n    #print(link.get('name'))\n    if link.get('href')!=None and link.get('href')[0]=='#':\n        Links.append(link.get('href')[1:])\nfor link in Links:\n    print(link)\n        \n    #remove the '#' and store them in a list\n"

In [13]:
#get section 
#example get section between Item 1 and Item 1A

#Problem also get the sentences
"""
Sections=""
for link in soup.find_all('a'):
    if link.has_attr('name') and link['name']=='ITEM_1_BUSINESS':
        for elements in link.next_elements:
            if elements.name=='a' and elements.has_attr('name') and elements['name']=='ITEM_1B_UNRESOLVED_STAFF_COMMENTS':
                break
                
            
            if str(elements)[0] != '<':
                Sections = Sections + str(elements)
                print(repr(elements))
            #Sections = Sections + str(elements)
            
        break
"""


'\nSections=""\nfor link in soup.find_all(\'a\'):\n    if link.has_attr(\'name\') and link[\'name\']==\'ITEM_1_BUSINESS\':\n        for elements in link.next_elements:\n            if elements.name==\'a\' and elements.has_attr(\'name\') and elements[\'name\']==\'ITEM_1B_UNRESOLVED_STAFF_COMMENTS\':\n                break\n                \n            \n            if str(elements)[0] != \'<\':\n                Sections = Sections + str(elements)\n                print(repr(elements))\n            #Sections = Sections + str(elements)\n            \n        break\n'